An example yaml config

```yaml
model:
  model_name: batch_reactor_test
  model_type: continuous
  constants:
    mu_m: 0.02
    K_m: 0.05
    K_i: 5.0
    v_par: 0.004
    Y_p: 1.2

  state_variables:
    X_s:
      init_val: 1.0
      init_val_lower: 0.9
      init_val_upper: 1.1
      rhs: mu_S*X_s - inp/V_s*X_s
      shape: (1,) # default value, can be omitted
      scaling: 1 # default value, can be omitted
    S_s:
      init_val: 0.5
      init_val_lower: 0.4
      init_val_upper: 0.6
      rhs: mu_S*X_s/Y_x - v_par*X_s/Y_p + inp/V_s*(S_in-S_s)
    P_s:
      init_val: 0.0
      init_val_lower: 0.0
      init_val_upper: 0.2
      rhs: v_par*X_s - inp/V_s*P_s
    V_s:
      init_val: 120.0
      init_val_lower: 110.0
      init_val_upper: 130.0
      rhs: inp

  control_variables:
    inp:
      shape: (1,) # default value, can be omitted

  user_defined_parameters:
    - Y_x
    - S_in
  aux_variables:
    mu_S:
      expr: mu_m*S_s/(K_m+S_s+(S_s**2/K_i))
```
---- 

# Define your model

In [1]:
from vis_utility import *

var_grid, widget_dict = var_num_grid_generator()

display(var_grid)


GridspecLayout(children=(Button(description='Model Specification', layout=Layout(grid_area='widget001', width=…

In [2]:
# define details of each variable/constant/parameter
num_constants = widget_dict["num_constants"].value
constant_grid, _widget_dict = constants_grid_generator(num_constants)
if num_constants > 0:
    display(constant_grid)
    widget_dict.update(_widget_dict)

GridspecLayout(children=(Button(description='Constants Definition', layout=Layout(grid_area='widget001', width…

In [3]:
"""
state_variables:
    X_s:
      init_val: 1.0
      init_val_lower: 0.9
      init_val_upper: 1.1
      rhs: mu_S*X_s - inp/V_s*X_s
      shape: (1,) # default value, can be omitted
      scaling: 1 # default value, can be omitted
"""
num_state_variables = widget_dict["num_state_variables"].value
sv_grid, _widget_dict = state_variables_grid_generator(num_state_variables)
if num_state_variables > 0:
    display(sv_grid)
    widget_dict.update(_widget_dict)
    
    

GridspecLayout(children=(Button(description='State Variables Definition', layout=Layout(grid_area='widget001',…

In [4]:
"""
control_variables:
    inp:
      shape: (1,) # default value, can be omitted
"""

num_control_variables = widget_dict["num_control_variables"].value
if num_control_variables > 0:
    control_grid, _widget_dict = control_variables_grid_generator(num_control_variables)
    display(control_grid)
    widget_dict.update(_widget_dict)



GridspecLayout(children=(Button(description='Control Variables Definition', layout=Layout(grid_area='widget001…

In [5]:
"""
  user_defined_parameters:
    - Y_x
    - S_in
"""
num_parameters = widget_dict["num_parameters"].value
if num_parameters > 0:
    parameter_grid, _widget_dict = parameters_grid_generator(num_parameters)
    display(parameter_grid)
    widget_dict.update(_widget_dict)


GridspecLayout(children=(Button(description='User Defined Parameter Definition', layout=Layout(grid_area='widg…

In [6]:
"""
aux_variables:
    mu_S:
      expr: mu_m*S_s/(K_m+S_s+(S_s**2/K_i))
"""

num_aux_variables = widget_dict["num_aux_variables"].value
if num_aux_variables > 0:
    aux_variable_grid, _widget_dict = aux_variable_grid_generator(num_aux_variables)
    display(aux_variable_grid)
    widget_dict.update(_widget_dict)
    





GridspecLayout(children=(Button(description='Auxiliary Variables Definition', layout=Layout(grid_area='widget0…

An example yaml config

```yaml

simulator:
  parameters:
    integration_tool: 'cvodes'
    abstol: 1.0e-10
    reltol: 1.0e-10
    t_step: 1.0
```
---- 

In [7]:
# define the simulator

num_simulator_parameters = BoundedIntText(value=1,
                                           min=0,
                                           step=1,
                                           description='Num. Simulator Parameters:',
                                           disabled=False)
display(num_simulator_parameters)

BoundedIntText(value=1, description='Num. Simulator Parameters:')

In [8]:
widget_dict.update({"num_simulator_parameters": num_simulator_parameters.value})

if num_simulator_parameters.value > 0:
    sim_para_grid, _widget_dict = simulator_parameter_grid_generator(num_simulator_parameters.value)
    display(sim_para_grid)
    widget_dict.update(_widget_dict)         

GridspecLayout(children=(Button(description='Simulator Parameters', layout=Layout(grid_area='widget001', width…

An example yaml config

```yaml
reward:
  step_reward:
    P_s:
      expr: P_s
      coef: 1.0
  terminal_reward:
    P_s:
      expr: P_s
      coef: 1.0
  input_reward:
    inp: 1.0
```
---- 

In [9]:
# define reward function

sv_names = [widget_dict[f"sv_{i}_def"].value for i in range(widget_dict["num_state_variables"].value)]
in_names = [widget_dict[f"cv_{i}_def"].value for i in range(widget_dict["num_control_variables"].value)]

state_rewards_grid, _widget_dict = state_rewards_grid_generator(sv_names)
if len(sv_names) > 0:
    widget_dict.update(_widget_dict)
    display(state_rewards_grid)
            

GridspecLayout(children=(Button(description='Define State Reward', layout=Layout(grid_area='widget001', width=…

In [10]:

input_rewards_grid, _widget_dict = input_rewards_grid_generator(in_names)
if len(in_names) > 0:
    widget_dict.update(_widget_dict)
    display(input_rewards_grid)
    
    


GridspecLayout(children=(Button(description='Define Input Reward', layout=Layout(grid_area='widget001', width=…

An example yaml config

```yaml
mpc:
  setup:
    n_horizon: 20
    n_robust: 0
    open_loop: 0
    t_step: 1.0
    state_discretization: collocation
    collocation_type: radau
    collocation_deg: 2
    collocation_ni: 2
    store_full_solution: True
  bounds:
    X_s:
      lower: 0.0 # required
      upper: 3.7 # required
      soft: False # default, can be omitted
    S_s:
      lower: -0.01
      upper: 2.0
    P_s:
      lower: 0.0
      upper: 3.0
    V_s:
      lower: 0.0
      upper: 150.0
    inp:
      lower: 0.0
      upper: 0.2
  uncertainities:
    Y_x: [0.5, 0.4, 0.3]
    S_in: [200.0, 220.0, 180.0]

```
---- 

An example yaml config

```yaml
estimator:
  type: StateFeedback # StateFeedback, EKF(Extended Kalman Filter), ..
  parameters: null
```
---- 